In [98]:
f = open("16.txt", "r")
lines = f.read().splitlines()

'''
{
    'name': 'AA',
    'rate': 10,
    'next': [a,a,a]
}
'''
def create_node(n, fr, neighbours):
    return {
        'name': n,
        'rate': fr,
        'next': neighbours,
        'open': False
    }

D = dict()

def init():
    for line in lines:
        ls = line.split()
        name = ls[1]
        rate = int(ls[4].split("=")[1][:-1])
        neighbours = list(map(lambda x: x if len(x) == 2 else x[:-1], ls[9:]))
        D[name] = create_node(name, rate, neighbours)
        # print(D[name])

In [99]:
from functools import cmp_to_key
def sort_nodes(nodes, pu=False):
    def cmp(a, b):
        if pu and D[a]['open'] and not D[b]['open']:
            return -1
        elif pu and not D[a]['open'] and D[b]['open']:
            return 1
        else:
            return D[a]['rate'] - D[b]['rate']
    return sorted(nodes, key=cmp_to_key(cmp), reverse=True)

def open_nodes(nodes):
    return set(n for n in nodes if D[n]['open'])

def unopen_nodes(nodes):
    return set(n for n in nodes if not D[n]['open'])

def h(current_pressure, unopen_nodes, time_left):
    sorted_nodes = sort_nodes(unopen_nodes)
    # print("sorted", sorted_nodes)
    while time_left - 2 > 0 and len(sorted_nodes) != 0:
        n, *sorted_nodes = sorted_nodes
        time_left = time_left - 2
        v = D[n]['rate'] * time_left
        current_pressure += v
        # print(n, v)
    return current_pressure

def h2(cp, un, tl):
    sn = sort_nodes(un)
    while tl - 2 > 0 and len(sn) != 0:
        tl -= 2
        n1, *sn = sn
        n2 = None
        if len(sn) > 0:
            n2, *sn = sn
        v1 = D[n1]['rate'] * tl
        v2 = D[n2]['rate'] * tl if n2 is not None else 0
        cp += v1 + v2
    return cp


# sort_nodes(['AA', 'BB', 'CC'])
    

In [100]:
p = [('AA', 'AA')]
best_p = p
mp = 0

def next_moves(node, m_left):
    if not node['open'] and node['rate'] > 0:
        yield node, (m_left-1)*node['rate']
    for n in sort_nodes(node['next'], pu=True):
        yield D[n], 0

def s(n, tp, ml):
    global mp

    unopen = unopen_nodes(D.keys())
    if (heur := h(tp, unopen, ml)) <= mp:
        return

    if ml == 0:
        if tp > mp:
            mp = tp
        return
    
    for next_node, add_pressure in next_moves(n, ml):
        n['open'] = (n['open'], True)[next_node['name'] == n['name']]
        s(next_node, tp+add_pressure, ml-1)
        n['open'] = (n['open'], False)[next_node['name'] == n['name']]

def s2(n1, n2, tp, ml):
    global mp, p, best_p
    
    unopen = unopen_nodes(D.keys())
    if (heur := h2(tp, unopen, ml)) <= mp:
        return

    if ml == 0:
        if tp > mp:
            best_p = p
            mp = tp
        return

    # cycle through every man move
    for nn1, ap1 in next_moves(n1, ml):
        # cycle through every elephant move
        n1['open'] = (n1['open'], True)[nn1['name'] == n1['name']]
        for nn2, ap2 in next_moves(n2, ml):
            n2['open'] = (n2['open'], True)[nn2['name'] == n2['name']]
            p = p + [(nn1['name'], nn2['name'])]
            # print(nn1['name'], nn2['name'])
            s2(nn1, nn2, tp + ap1 + ap2, ml-1)
            *p, _ = p
            n2['open'] = (n2['open'], False)[nn2['name'] == n2['name']]
        n1['open'] = (n1['open'], False)[nn1['name'] == n1['name']]

    

In [96]:
init()
mp=0
s(D['AA'], 0, 30)
print(mp)

1650


In [101]:
init()
mp=0
p = [('AA', 'AA')]
s2(D['AA'], D['AA'], 0, 26)
print(mp)
print(best_p)

KeyboardInterrupt: 

In [45]:
p = ['AA']
best_p = p
mp = 0
def search(node, total_pressure, m_left, prev):
    global p, mp

    unopen = unopen_nodes(D.keys())
    if (heur := h(total_pressure, unopen, m_left)) <= mp: # skip if est. value is less than current best
        # print('skipping', p, heur)
        return

    if m_left == 0:
        if total_pressure > mp:
            mp = total_pressure
            best_p = p
            print(total_pressure)
        return
        
    if not node['open'] and node['rate'] > 0: # try with opening only here
        node['open'] = True
        add_pressure = (m_left-1)*node['rate']
        search(node, total_pressure+add_pressure, m_left-1, None)
        node['open'] = False
    
    # try all possible moves
    # prioritize unopened and with higher rate
    next_nodes = sort_nodes(node['next'], pu=True)
    # stop immediate cycling
    # if prev is not None and prev in next_nodes:
    #     next_nodes.remove(prev)
    for n in next_nodes:
        p = p + [n]
        search(D[n], total_pressure, m_left-1, node)
        *p, _ = p

In [46]:
search(D['AA'], 0, 30, None)

1122
1126
1156
1163
1186
1193
1206
1258
1274
1330
1346
1402
1418
1474
1490
1546
1562
1582
1631
1638
1642
1646
1650
